# Day 11: 使用 Weights & Biases (WandB) 进行实验追踪

在深度学习实验中，记录和可视化训练过程（Loss, Accuracy, 显存占用等）至关重要。**Weights & Biases (WandB)** 是目前最流行的实验追踪工具之一，它比 TensorBoard 更强大，支持云端存储、多人协作、超参搜索等功能。

**学习目标：**
- 掌握 WandB 的基本配置与初始化 (`wandb.init`)
- 学会记录指标 (`wandb.log`)
- 将 WandB 集成到 PyTorch 训练循环中
- 将 WandB 集成到 Hugging Face Trainer 中

**前置准备：**
1. 注册 WandB 账号: [https://wandb.ai/](https://wandb.ai/)
2. 获取 API Key: [https://wandb.ai/authorize](https://wandb.ai/authorize)
3. 安装库: `pip install wandb`

In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random
import numpy as np

## 1. 登录 WandB

在 Notebook 中运行以下代码进行登录。如果是第一次运行，会提示输入 API Key。

In [ ]:
wandb.login()
# 如果已经登录过，会自动读取本地配置

## 2. WandB 基础概念

- **Project**: 项目，对应一个具体的任务（例如 "cifar10-classification"）。
- **Run**: 运行，对应一次具体的实验（例如 "resnet18-lr0.01"）。
- **Config**: 超参数配置，记录本次实验的设置（LR, Batch Size, Optimizer 等）。
- **Log**: 记录随时间变化的指标（Loss, Accuracy）。

## 3. 集成到 PyTorch 训练循环

我们将使用一个简单的 MLP 在 MNIST 上演示 WandB 的集成。

In [ ]:
# 1. 定义超参数配置
config = {
    "learning_rate": 0.01,
    "epochs": 5,
    "batch_size": 64,
    "architecture": "MLP",
    "dataset": "MNIST"
}

In [ ]:
def train_model():
    # 2. 初始化 WandB Run
    # with wandb.init(...) 可以在代码块结束时自动 finish，非常推荐
    with wandb.init(project="handonDL-day11-demo", config=config):
        
        # 通过 wandb.config 访问超参数（这样如果使用 sweep 超参搜索，wandb 会自动注入参数）
        cfg = wandb.config
        
        # 数据准备
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
        
        # 模型定义
        model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
        optimizer = optim.SGD(model.parameters(), lr=cfg.learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        # 3. 监控模型梯度和参数（可选）
        wandb.watch(model, log="all", log_freq=100)
        
        print("Start Training...")
        for epoch in range(cfg.epochs):
            model.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                if batch_idx % 100 == 0:
                    # 4. 记录指标
                    wandb.log({"epoch": epoch, "loss": loss.item()})
            
            # 模拟验证集准确率
            val_acc = 0.9 + 0.01 * epoch + random.uniform(-0.005, 0.005)
            wandb.log({"val_acc": val_acc})
            print(f"Epoch {epoch}: Val Acc {val_acc:.4f}")
            
    # with 语句结束时自动调用 wandb.finish()

In [ ]:
# 运行训练
train_model()

## 4. 集成到 Hugging Face Trainer

Hugging Face 的 `Trainer` 原生支持 WandB，集成非常简单，只需要在 `TrainingArguments` 中设置 `report_to="wandb"`。

In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer

# 示例代码（不实际运行，因为需要下载大模型）
def hf_integration_demo():
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        logging_steps=10,
        
        # 关键参数
        report_to="wandb",  # 启用 WandB 报告
        run_name="bert-finetune-demo" # 设置 WandB Run 的名称
    )
    
    # model = ...
    # trainer = Trainer(
    #     model=model,
    #     args=training_args,
    #     ...
    # )
    # trainer.train()
    
    print("Hugging Face Trainer 集成示例：设置 report_to='wandb' 即可。")

hf_integration_demo()

## 5. 进阶功能

- **Artifacts**: 用于版本化管理数据集和模型文件。`run.log_artifact(artifact_obj)`
- **Sweeps**: 自动化超参数搜索。WandB 提供了一个强大的 Sweep Controller，可以自动尝试不同的超参组合。
- **Tables**: 记录表格数据（如文本生成结果、图片预测结果），支持在 Web 界面进行交互式分析。

## 6. 总结

WandB 是现代深度学习工作流中不可或缺的工具。相比于本地的日志文件，它提供了：
1. **实时可视化**：无需等待训练结束。
2. **集中管理**：所有实验记录在云端，不再丢失。
3. **对比分析**：轻松对比不同 Run 之间的效果。

建议在日后的所有实验中都开启 WandB。